In [4]:
%load_ext tensorboard
%tensorboard --logdir ./result

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 59905), started 0:02:26 ago. (Use '!kill 59905' to kill it.)

In [5]:
import os
import sys
sys.path.insert(0, '/rl')

In [6]:
import math
import random
from typing import List, Optional, Tuple, Union
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn
import os
import gc
from torch.utils.data import IterableDataset, Dataset
import json
import numpy as np
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PretrainedConfig
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DefaultDataCollator, DataCollatorForTokenClassification, AutoConfig

In [7]:
SYSTEM_PROMPT = """
    Use below format to answer questions
    <think>
    your thinking process
    </think>
    <answer>
    your answer
    </answer>
    """

In [8]:
gc.collect()
torch.cuda.empty_cache()

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [28]:
# eval result
# query = "北京在哪个国家？"
query = "there are 2 chicken and 3 fish and 4 birds, how many ocean animals in total?"
org_model = AutoModelForCausalLM.from_pretrained("../Qwen2.5-1.5B-Instruct", local_files_only=True).to(device)
opt_model = AutoModelForCausalLM.from_pretrained("./result/checkpoint_4000", local_files_only=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("../Qwen2.5-1.5B-Instruct", local_files_only=True)

In [29]:
input_text = tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": query},
                ],
                add_generation_prompt=True,
                tokenize=False,
            )
input_text

'<|im_start|>system\n\n    Use below format to answer questions\n    <think>\n    your thinking process\n    </think>\n    <answer>\n    your answer\n    </answer>\n    <|im_end|>\n<|im_start|>user\nthere are 2 chicken and 3 fish and 4 birds, how many ocean animals in total?<|im_end|>\n<|im_start|>assistant\n'

In [30]:
model_inputs = tokenizer([input_text], return_tensors="pt").to(org_model.device)

generated_ids = org_model.generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

'To find the total number of ocean animals, we need to add up all the different types of animals.\n\nThink: We have chickens, fish, and birds. The question asks for "ocean animals," so let\'s count each type separately.\n1. Chickens: There are 2 chickens.\n2. Fish: There are 3 fish.\n3. Birds: There are 4 birds.\n\nNow, let\'s add them together:\n2 (chickens) + 3 (fish) + 4 (birds) = 9 ocean animals in total.\n\nAnswer: There are 9 ocean animals in total.'

In [31]:
generated_ids = opt_model.generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

'<think>\nTo find the total number of ocean animals, we need to add up all the different types of animals mentioned.\n</think>\n<answer>\nThe total number of ocean animals is 2 + 3 + 4 = 9.\n</answer>'